Backlog
--

Process backlog data to set up files for normal processing

In [ ]:
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

import pandas as pd

%matplotlib inline

In [ ]:
def daywave(date, net, sta, loc, cmp):
  global st
  time = UTCDateTime(date)
  st = client.get_waveforms(net, sta, loc, cmp, time, time + 86400, attach_response=True)

In [ ]:
client = Client("http://service.geonet.org.nz")

In [ ]:
names = ['date', 'value', 'error']
dt1str = '2009-05-19'
dt2str = '2016-09-30'

In [ ]:
sta = 'GLKZ'
#chan = [{'loc':'40', 'cmp':'LTH', 'net':'NZ', 'fname':'gll.csv', 'subject':'green lake level'},
        #{'loc':'81', 'cmp':'LKO', 'net':'NZ', 'fname':'mbt.csv', 'subject':'marker bay temperature'},
        #{'loc':'80', 'cmp':'LKO', 'net':'NZ', 'fname':'glt.csv', 'subject':'green lake temperature'}]
#chan = [{'loc':'40', 'cmp':'LTH', 'net':'NZ', 'fname':'gll.csv', 'subject':'green lake level'}]
#chan = [{'loc':'80', 'cmp':'LKO', 'net':'NZ', 'fname':'glt.csv', 'subject':'green lake temperature'}]
chan = [{'loc':'81', 'cmp':'LKO', 'net':'NZ', 'fname':'mbt.csv', 'subject':'marker bay temperature'}]

In [ ]:
for ch in chan:
    cmp = ch['cmp']
    net = ch['net']
    fname = ch['fname']
    loc = ch['loc']
    subject = ch['subject']
    print fname, cmp, loc, subject

    df = pd.DataFrame()
    for date in pd.date_range(start=dt1str, end=dt2str, freq='D'):
        #print date.date()
        try:
            daywave(date, net, sta, loc, cmp)
            st.remove_sensitivity()
            st.merge(fill_value = 'interpolate')
            tr = st[0]
            mean = tr.data.mean()
            std = tr.data.std()
        except:
            print date.date(), 'retrieve unsuccessful'
            mean = float('nan')
            std = float('nan')
        print date.date(), mean, std
        vals = {'date':date.date(), 'value':mean, 'error':std}
        df = df.append(pd.DataFrame([vals], columns=vals.keys()), ignore_index=True)
    print

    df.to_csv(fname, columns=['date', 'value', 'error'], na_rep='NaN', index=False)

**Check**

In [ ]:
oldnames = ['d1', 'date', 't1', 'value']

**GLT**

In [ ]:
old = pd.read_csv('glt.dat', sep='\s+', parse_dates=True, skiprows=0, names=oldnames, index_col=0)
new = pd.read_csv('glt.csv', parse_dates=True, index_col=0)

In [ ]:
old.head()

In [ ]:
p = new['value'].plot(label='new')
old['value'].plot(ax=p, label='old')
p.legend(loc='best')

**MBT**

In [ ]:
old = pd.read_csv('mbt.dat', sep='\s+', parse_dates=True, skiprows=0, names=names, index_col=0)
new = pd.read_csv('mbt.csv', parse_dates=True, index_col=0)

In [ ]:
old.head()

In [ ]:
p = new['value'].plot(label='new')
old['value'].plot(ax=p, label='old')
p.legend(loc='best')

**GLL**

In [ ]:
old = pd.read_csv('gll.dat', sep='\s+', parse_dates=True, skiprows=0, names=oldnames, index_col=1)
new = pd.read_csv('gll.csv', parse_dates=True, index_col=0)

In [ ]:
old.tail()

In [ ]:
new.tail()

In [ ]:
p = new['value'].plot(label='new')
old['value'].plot(ax=p, label='old')
p.legend(loc='best')